In [2]:
import pandas as pd

In [3]:
reviews_df = pd.read_csv('steam_review_data.csv')


In [4]:
games_df = pd.read_csv('steamspy_data.csv')


In [5]:
# Filter out games with less than 100 reviews
games_df = games_df[(games_df['positive'] + games_df['negative']) >= 100]

In [6]:
games_df['price'].fillna(0, inplace=True)

In [7]:
games_df = games_df.drop(columns=['score_rank', 'userscore', 'initialprice', 'discount'])
reviews_df = reviews_df.drop(columns=['Unnamed: 0', 'recommendationid', 'author', 'author_playtime_last_two_weeks', 'author_last_played', 'author_num_reviews', 'votes_funny', 'weighted_vote_score', 'timestamp_updated', 'steam_purchase', 'received_for_free', 'written_during_early_access'])


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [29]:
games_df = pd.read_csv('steamspy_data.csv')
reviews_df = pd.read_csv('steam_review_data.csv')

merged_df = pd.merge(games_df, reviews_df, on='appid')


In [32]:
merged_df.head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,...,votes_up,votes_funny,weighted_vote_score,comment_count,timestamp_created,timestamp_updated,steam_purchase,received_for_free,written_during_early_access,review
0,10,Counter-Strike,Valve,Valve,NaN,193046,4940,0,"10,000,000 .. 20,000,000",8486,...,405,0,0.976047,1,1460808481,1460808481,False,False,False,Best FPS game i ever played
1,10,Counter-Strike,Valve,Valve,NaN,193046,4940,0,"10,000,000 .. 20,000,000",8486,...,428,284,0.953599,20,1495993232,1499801573,True,False,False,How to correctly play this game:\n-Noisiest fu...
2,10,Counter-Strike,Valve,Valve,NaN,193046,4940,0,"10,000,000 .. 20,000,000",8486,...,248,9,0.950770,220,1480429968,1653066625,True,False,False,[h1]⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ ⠀⠀ ♡[/h1]
3,10,Counter-Strike,Valve,Valve,NaN,193046,4940,0,"10,000,000 .. 20,000,000",8486,...,2487,4130,0.947676,120,1427890049,1480048737,False,False,False,Ruined my life.
4,10,Counter-Strike,Valve,Valve,NaN,193046,4940,0,"10,000,000 .. 20,000,000",8486,...,195,0,0.938016,9,1357415282,1547414438,False,False,False,Actually the best game in this world. It still...


In [36]:
merged_df['sentiment_ratio'] = merged_df.apply(lambda row: row['positive'] / row['negative'] if row['negative'] != 0 else float('inf'), axis=1)


In [37]:
average_sentiment = merged_df.groupby('appid')['sentiment_ratio'].mean()

In [45]:
merged_df[merged_df['appid'] == 10]


,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,...,votes_funny,weighted_vote_score,comment_count,timestamp_created,timestamp_updated,steam_purchase,received_for_free,written_during_early_access,review,sentiment_ratio
0,10,Counter-Strike,Valve,Valve,NaN,193046,4940,0,"10,000,000 .. 20,000,000",8486,...,0,0.976047,1,1460808481,1460808481,False,False,False,Best FPS game i ever played,39.078138
1,10,Counter-Strike,Valve,Valve,NaN,193046,4940,0,"10,000,000 .. 20,000,000",8486,...,284,0.953599,20,1495993232,1499801573,True,False,False,How to correctly play this game:\n-Noisiest fu...,39.078138
2,10,Counter-Strike,Valve,Valve,NaN,193046,4940,0,"10,000,000 .. 20,000,000",8486,...,9,0.950770,220,1480429968,1653066625,True,False,False,[h1]⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ ⠀⠀ ♡[/h1],39.078138
3,10,Counter-Strike,Valve,Valve,NaN,193046,4940,0,"10,000,000 .. 20,000,000",8486,...,4130,0.947676,120,1427890049,1480048737,False,False,False,Ruined my life.,39.078138
4,10,Counter-Strike,Valve,Valve,NaN,193046,4940,0,"10,000,000 .. 20,000,000",8486,...,0,0.938016,9,1357415282,1547414438,False,False,False,Actually the best game in this world. It still...,39.078138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,10,Counter-Strike,Valve,Valve,NaN,193046,4940,0,"10,000,000 .. 20,000,000",8486,...,3,0.816768,0,1686340166,1686340166,True,False,False,[spoiler]Old is Gold. <3[/spoiler],39.078138
96,10,Counter-Strike,Valve,Valve,NaN,193046,4940,0,"10,000,000 .. 20,000,000",8486,...,1,0.816473,3,1607023925,1607079793,True,False,False,It's a classic game. I heard about it from fri...,39.078138
97,10,Counter-Strike,Valve,Valve,NaN,193046,4940,0,"10,000,000 .. 20,000,000",8486,...,48,0.816323,0,1641675510,1651369760,True,False,False,Counter-Strike is good game,39.078138
98,10,Counter-Strike,Valve,Valve,NaN,193046,4940,0,"10,000,000 .. 20,000,000",8486,...,218,0.815739,4,1448051170,1448051170,False,False,False,Playing it since 2010.\nStill bad at it.\n\n\n...,39.078138


In [35]:
def calculate_sentiment(row):
    if row['negative'] == 0:
        if row['positive'] > 0:
            return 'Extremely Positive'
        else:
            return 'No Reviews'
    else:
        return row['positive'] / row['negative']

games_df['sentiment'] = games_df.apply(calculate_sentiment, axis=1)

# Filter the DataFrame to exclude rows where 'sentiment' is 'Extremely Positive'
games_df = games_df[games_df['sentiment'] != 'Extremely Positive']



In [38]:
# Select features and target
features = ['developer', 'genre', 'price']  # Add 'categories' if available
X = games_df[features]
y = games_df['sentiment']

# Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['price']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['developer', 'genre'])
    ])


# Model pipeline
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', RandomForestRegressor())])

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))


Mean Squared Error: 148.36342857226273


In [40]:
games_df['sentiment'].mean()

8.41945002451485